In [7]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from joblib import dump, load

In [9]:
input_x_train_file = "drugs_x_test.csv"
input_y_train_file = "drugs_y_train.csv"

df_x = pd.read_csv(input_x_train_file)
df_y = pd.read_csv(input_y_train_file)

x = df_x.values
y = df_y.values

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2435103608.py, line 1)

In [ ]:
clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(x, y)

NameError: name 'x' is not defined

In [ ]:
model_file = "mlp.joblib"

dump(clf, model_file)